In [13]:
#Libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv
from googlesearch import search

In [ ]:
def obtener_titulos_peliculas():
    url = "https://trends.google.com/trends/trendingsearches/daily?geo=US&hl=es"

    # Configurar Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    # Obtener la página con Selenium
    driver.get(url)

    try:
        # Esperar a que la página se cargue completamente (puedes ajustar el tiempo si es necesario)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[ng-repeat="titlePart in titleArray"]'))
        )

        # Intentar hacer clic en el botón "Cargar más" varias veces
        for _ in range(5):  # Puedes ajustar la cantidad de clics según sea necesario
            cargar_mas_button = driver.find_element(By.CLASS_NAME, 'feed-load-more-button')
            if cargar_mas_button.is_displayed():
                driver.execute_script("arguments[0].click();", cargar_mas_button)
                time.sleep(2)  # Esperar a que carguen los nuevos resultados
            else:
                break  # Si el botón no está visible, salimos del bucle

        # Obtener el HTML después de cargar completamente
        page_source = driver.page_source

        # Analizar el HTML de la página
        soup = BeautifulSoup(page_source, 'html.parser')

        # Encontrar los títulos de las películas
        titulos = soup.select('[ng-repeat="titlePart in titleArray"]')

        for indice, titulo in enumerate(titulos, start=1):
            print(f"{indice}. {titulo.text.strip()}")

    finally:
        # Cerrar el navegador
        driver.quit()

if __name__ == "__main__":
    obtener_titulos_peliculas()


CARGA TRENDS & CATEGORIA EN UN CSV

In [12]:
# Configurar Selenium
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

def obtener_categoria_google(query):
    try:
        resultados = search(query, num=1, stop=1, pause=2)
        for j in resultados:
            soup = BeautifulSoup(j, 'html.parser')
            categoria = soup.find('meta', {'property': 'og:category'})
            if categoria:
                return categoria['content']
    except Exception as e:
        print(f"Error al obtener la categoría para '{query}': {e}")
    return "No se encontró categoría"

def obtener_titulos_peliculas():
    url_trends = "https://trends.google.com/trends/trendingsearches/daily?geo=US&hl=es"
    
    # Obtener la página con Selenium
    driver.get(url_trends)

    # Esperar a que la página se cargue completamente
    time.sleep(5)  # Puedes ajustar este tiempo según sea necesario

    # Obtener el HTML después de cargar completamente
    page_source = driver.page_source

    # Analizar el HTML de la página
    soup = BeautifulSoup(page_source, 'html.parser')

    # Encontrar los títulos de las películas
    titulos = soup.select('[ng-repeat="titlePart in titleArray"]')

    # Crear una lista para almacenar los datos
    datos = []

    for indice, titulo_element in enumerate(titulos, start=1):
        titulo = titulo_element.text.strip()
        categoria = obtener_categoria_google(titulo)
        print(f"{indice}. Título: {titulo}, Categoría: {categoria}")

        # Agregar los datos a la lista
        datos.append([titulo, categoria])

    # Guardar los datos en un archivo CSV
    with open('resultados.csv', mode='w', newline='', encoding='utf-8') as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        escritor_csv.writerow(['Título', 'Categoría'])  # Escribir encabezados
        escritor_csv.writerows(datos)

if __name__ == "__main__":
    obtener_titulos_peliculas()


Error al obtener la categoría para 'India vs New Zealand': search() got an unexpected keyword argument 'num'
1. Título: India vs New Zealand, Categoría: No se encontró categoría
Error al obtener la categoría para 'Kevin McCarthy': search() got an unexpected keyword argument 'num'
2. Título: Kevin McCarthy, Categoría: No se encontró categoría
Error al obtener la categoría para 'Matt Petgrave': search() got an unexpected keyword argument 'num'
3. Título: Matt Petgrave, Categoría: No se encontró categoría
Error al obtener la categoría para 'Warriors': search() got an unexpected keyword argument 'num'
4. Título: Warriors, Categoría: No se encontró categoría
Error al obtener la categoría para 'Markwayne Mullin': search() got an unexpected keyword argument 'num'
5. Título: Markwayne Mullin, Categoría: No se encontró categoría
Error al obtener la categoría para '76ers': search() got an unexpected keyword argument 'num'
6. Título: 76ers, Categoría: No se encontró categoría
Error al obtener la 